This notebook create a GUI to identify the room doors and dimensions.

Then, it turns the data into clean JSONs

## Use of the GUI

### Initialisation

Choosing in the variable _path_ between map chunk a, b, c or d

### Commands
* left click : mark a corner or a door extremity
* right arrow (on keyboard) : change between Corner Mode and Door Mode (default is Corner Mode)

### For each room
* 1) Write the room name in the upper textbox and every connected room door in the lower, separated by commas + space, in the order in which the door will be registered
* 2) Click on each corner of the room in Corner Mode and each extremity of each door in Door Mode beginning at the north-west corner and strictly following the walls in the negative direction. In other word : NW --> NE --> SE --> SW

Disclaimer : this GUI was not designed to be used outside of the project development process. It is not user-friendly.

In [ ]:
import pandas as pd

import PIL.Image

from tkinter import *

import ast

In [ ]:
path = "Louvre_map/Louvre_map_c.png"

im = PIL.Image.open(path)
width, height = im.size
#im.thumbnail((int(0.5*width),int(0.5*height)))
#width, height = im.size

In [ ]:
# A faire : ajouter une boite de sélection pour le choix de la salle et de toutes les portes (mettre plusieurs boites)
# Ajouter la possibilité d'annuler la création de salle en cours
# Ou alors contrôler dans le nettoyage des données que les salles ont le bon nom et simplement placer une liste
# indicative ici ?

#On notera qu'à cause du traitement de l'OCR, le numéro des salles des meubles XVIII est perdu !
valid_rooms = []
with open("Rooms.txt","r",encoding = "utf-8") as f:
    for line in f:
        valid_rooms.append(line.replace("é","e").split("/")[0].replace("\n",""))

In [ ]:
corner = 0
door = False
ongoing_door = False
door_first_coord = None
door_counter = 0

x = 0
y = 0

old_x = 0
old_y = 0

anchor_x = 0
anchor_y = 0

rooms = []
room = {}

r = 3

window = Tk()

canvas = Canvas(window,width=width,height=height)

louvre = PhotoImage(master = canvas,file=path)

def init_image():
    canvas.create_image(0, 0, image=louvre, anchor=NW)

init_image()

roomBox=Text(window, height=1, width=50)
roomBox.pack()

doorBox=Text(window, height=1, width=100)
doorBox.pack()

roomsLabel = Label(window, text = " - ".join(valid_rooms),wraplength = width-200)
roomsLabel.pack()

def draw_circle(X,Y,color):
    canvas.create_oval(X-r,Y-r,X+r,Y+r,fill = color)
    
def create_door(X,Y):
    """
    Called when a click is made in door mode. Register the points and group them by pairs to form doors.
    """
    global ongoing_door
    global room
    global door_first_coord
    global anchor_x
    global anchor_y
    global door_counter
    print("Creating_door")
    if ongoing_door:
        doorText = doorBox.get("1.0","end-1c").split(", ")[door_counter]
        door_counter += 1
        room["doors"].append((doorText,door_first_coord,(X,Y)))
        ongoing_door = False
    else:
        door_first_coord = (X,Y)
        ongoing_door = True
    draw_circle(anchor_x + X,anchor_y + Y,"blue")

def callback(event):
    """
    Main function identifying the category and the cardinal direction of each clicking event,
    and correctly displaying and registering it.
    """
    print("clicked at", event.x, event.y)
    global room
    global rooms
    global corner
    global x
    global y
    global old_x
    global old_y
    global anchor_x
    global anchor_y
    global door_counter
    if corner == 0:
        if door:
            x = 0
            y = min(max(event.y - anchor_y,0),old_y)
            create_door(x,y)
        else:
            print("Creating new room")
            if room:
                rooms.append(room)
            anchor_x = event.x
            anchor_y = event.y
            x = 0
            y = 0
            old_x = 0
            old_y = 0
            room = {"name":roomBox.get("1.0","end-1c"),"doors":[],"NO":(x,y)}
            init_image()
            draw_circle(x+anchor_x,y+anchor_y,"red")
            corner += 1
            door_counter = 0
    
    elif corner == 1:
        x = max(event.x-anchor_x,old_x)
        y = old_y
        if door:
            create_door(max(x,0),y)
        else:
            room["NE"] = (x,y)
            draw_circle(x+anchor_x,y+anchor_y,"red")
            corner += 1
    
    elif corner == 2:
        x = old_x
        y = max(event.y-anchor_y,old_y)
        if door:
            create_door(x,max(y,0))
        else:
            room["SE"] = (x,y)
            draw_circle(x+anchor_x,y+anchor_y,"red")
            corner += 1
    
    else:
        x = max(min(event.x-anchor_x,old_x),0)
        y = old_y
        if door:
            create_door(x,y)
        else:
            room["SO"] = (0,y)
            draw_circle(anchor_x,y+anchor_y,"red")
            corner += 1
        
    old_x = x
    old_y = y
    corner = corner%4

                    
def door_change(event):
    global door
    door = not door 
    
canvas.bind("<Button-1>", callback)
window.bind("<Right>",door_change)
canvas.pack()

window.mainloop()

In [ ]:
print(rooms)

rooms_df = pd.DataFrame(rooms)

In [ ]:
import os

if not os.path.exists('data/rooms.csv'):
    rooms_df.to_csv("data/rooms.csv")
else:
    with open('data/rooms.csv', 'a') as f:
        rooms_df.to_csv(f, header=False)


## File formatting

In [ ]:
#Real scale is 1/4, but the scaling is done in Unity
def create_wall(row,target,corner_1,corner_2, scale = 1):
        
    dic = {}
    
    dic["card"] = target
    p00, p01 = row[corner_1].strip(")(").split(", ")
    p10, p11 = row[corner_2].strip(")(").split(", ")
       
    dic["p00"], dic["p01"], dic["p10"], dic["p11"] = scale*int(p00), scale*int(p01), scale*int(p10), scale*int(p11)
    
    return dic

def create_doors(text, scale = 1):
    l = ast.literal_eval(text)
    
    result = []
    
    for door in l:
        
        dic = {}
        
        dic["name"] = door[0].strip("'")
        p00, p01 = door[1][0], door[1][1]
        p10, p11 = door[2][0], door[2][1]
        
        dic["p00"], dic["p01"], dic["p10"], dic["p11"] = scale*int(p00), scale*int(p01), scale*int(p10), scale*int(p11)
        
        result.append(dic)
    
    return result
   

In [ ]:
full_rooms_df = pd.read_csv("data/rooms.csv", encoding = "utf-8")
full_rooms_df

In [ ]:
for index, row in full_rooms_df.iterrows():
    
    walls = [create_wall(row,"n","NO","NE"),create_wall(row,"e","NE","SE"),\
             create_wall(row,"s","SE","SO"),create_wall(row,"o","SO","NO")]    
    walls_df = pd.DataFrame(walls)
    
    doors_df = pd.DataFrame(create_doors(row.doors))
    
    walls_df.to_json("data/walls_json/{}.json".format(row["name"]), orient="records")
    doors_df.to_json("data/doors_json/{}.json".format(row["name"]), orient="records")